In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Install MTCNN 
to detect face install MTCNN.  “Multi-Task Cascaded Convolutional Neural Network,” or MTCNN is one of the more popular approaches for Face Detection With Deep Learning

In [1]:
!pip install mtcnn

  Created wheel for mtcnn: filename=mtcnn-0.0.9-cp37-none-any.whl size=2257698 sha256=8465fbd5d25077bac53265c9bac6c85a4a33f96d549de902e37c7bd2b07925e1
  Stored in directory: C:\Users\user\AppData\Local\pip\Cache\wheels\85\81\65\6363fa5aafd7a155c896591e0c7c6e27b69642aa82b9cbf076
Successfully built mtcnn


In [2]:
!pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to c:\users\user\appdata\local\temp\pip-req-build-o7b4_k86


  Running command git clone -q https://github.com/rcmalli/keras-vggface.git 'C:\Users\user\AppData\Local\Temp\pip-req-build-o7b4_k86'
  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git clone -q https://github.com/rcmalli/keras-vggface.git 'C:\Users\user\AppData\Local\Temp\pip-req-build-o7b4_k86'
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [11]:
#import all neccessary dependecies

import numpy as np
import cv2
import os
import tensorflow as tf
from keras.models import Model, Sequential, model_from_json
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, RMSprop
from keras.models import load_model
from mtcnn.mtcnn import MTCNN
from os import walk
from os import listdir
from os.path import join

In [2]:
detector = MTCNN()






Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.



In [3]:
class_names=np.array(['Scarlett Johansson', 'Willa Holland', 'alexandra daddario', 'amber heard face', 'anne hathaway', 'kaushal'])
class_names[3]

'amber heard face'

In [0]:
batch_size = 20
epochs = 2
steps_per_epoch = 100

In [7]:
#ImageDataGenerator for traning

train_dir = '/content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/train'
test_dir = '/content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/test'

input_shape=(224,224)
datagen_train = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      vertical_flip=True,
      fill_mode='nearest')

datagen_test = ImageDataGenerator(rescale=1./255)

generator_train = datagen_train.flow_from_directory(directory=train_dir,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    shuffle=True)
generator_test = datagen_test.flow_from_directory(directory=test_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False)

steps_test = generator_test.n / batch_size
num_classes = generator_train.num_classes
cls_train = generator_train.classes
cls_test = generator_test.classes

Found 487 images belonging to 6 classes.
Found 147 images belonging to 6 classes.


In [0]:
def path_join(dirname, filenames):
    return [os.path.join(dirname, filename) for filename in filenames]

In [9]:
steps_test = generator_test.n / batch_size
image_paths_train = path_join(train_dir, generator_train.filenames)
image_paths_test = path_join(test_dir, generator_test.filenames)
cls_train = generator_train.classes
cls_test = generator_test.classes
class_names = list(generator_train.class_indices.keys())
print('class name',class_names)
num_classes = generator_train.num_classes
print('num classes',num_classes)

class name ['Scarlett Johansson', 'Willa Holland', 'alexandra daddario', 'amber heard face', 'anne hathaway', 'kaushal']
num classes 6


In [82]:
#make VGG-Face Sequential() model

def loadVggFaceModel():
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Convolution2D(4096, (7, 7), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(4096, (1, 1), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(2622, (1, 1)))
    model.add(Flatten())
    model.add(Activation('softmax'))
    model.load_weights('/content/drive/My Drive/Colab Notebooks/face recognition/vgg_face_weights.h5')
    loaded_model = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)
#     predictions = Dense(6, activation = 'softmax')(loaded_model.output)
#     new_model = Model(input = loaded_model.input, output = predictions)
    return loaded_model

vgg_model = loadVggFaceModel()
vgg_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_14_input (Inp (None, 224, 224, 3)       0         
_________________________________________________________________
zero_padding2d_14 (ZeroPaddi (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_15 (ZeroPaddi (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_16 (ZeroPaddi (None, 114, 114, 64)      0   

In [0]:
#customize the model
new_model = Sequential()
new_model.add(vgg_model)
new_model.add(Dense(1024, activation='relu'))
new_model.add(Dropout(.5))
new_model.add(Dense(num_classes,activation='softmax'))

In [0]:
#hyper parameters for traning
optimizer = Adam(lr=1e-5)
loss = 'categorical_crossentropy'
metrics = ['categorical_accuracy']

In [85]:
#to check if layer of a model is trainable or not
def print_layer_trainable(model):
    for layer in model.layers:
        print("{0}:\t{1}".format(layer.trainable, layer.name))
        
print_layer_trainable(vgg_model)

False:	zero_padding2d_14_input
True:	zero_padding2d_14
True:	conv2d_17
True:	zero_padding2d_15
True:	conv2d_18
True:	max_pooling2d_6
True:	zero_padding2d_16
True:	conv2d_19
True:	zero_padding2d_17
True:	conv2d_20
True:	max_pooling2d_7
True:	zero_padding2d_18
True:	conv2d_21
True:	zero_padding2d_19
True:	conv2d_22
True:	zero_padding2d_20
True:	conv2d_23
True:	max_pooling2d_8
True:	zero_padding2d_21
True:	conv2d_24
True:	zero_padding2d_22
True:	conv2d_25
True:	zero_padding2d_23
True:	conv2d_26
True:	max_pooling2d_9
True:	zero_padding2d_24
True:	conv2d_27
True:	zero_padding2d_25
True:	conv2d_28
True:	zero_padding2d_26
True:	conv2d_29
True:	max_pooling2d_10
True:	conv2d_30
True:	dropout_4
True:	conv2d_31
True:	dropout_5
True:	conv2d_32
True:	flatten_2


In [86]:
vgg_model.trainable = False #i dont know why it didnt work
for layer in vgg_model.layers:
    layer.trainable = False
print_layer_trainable(new_model)
print('new model')
print_layer_trainable(vgg_model)

False:	model_2
True:	dense_3
True:	dropout_6
True:	dense_4
new model
False:	zero_padding2d_14_input
False:	zero_padding2d_14
False:	conv2d_17
False:	zero_padding2d_15
False:	conv2d_18
False:	max_pooling2d_6
False:	zero_padding2d_16
False:	conv2d_19
False:	zero_padding2d_17
False:	conv2d_20
False:	max_pooling2d_7
False:	zero_padding2d_18
False:	conv2d_21
False:	zero_padding2d_19
False:	conv2d_22
False:	zero_padding2d_20
False:	conv2d_23
False:	max_pooling2d_8
False:	zero_padding2d_21
False:	conv2d_24
False:	zero_padding2d_22
False:	conv2d_25
False:	zero_padding2d_23
False:	conv2d_26
False:	max_pooling2d_9
False:	zero_padding2d_24
False:	conv2d_27
False:	zero_padding2d_25
False:	conv2d_28
False:	zero_padding2d_26
False:	conv2d_29
False:	max_pooling2d_10
False:	conv2d_30
False:	dropout_4
False:	conv2d_31
False:	dropout_5
False:	conv2d_32
False:	flatten_2


In [0]:
#compile the new custom model
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [10]:
#we dont have a balanced dataset so we can get weights for each seprate class
from sklearn.utils.class_weight import compute_class_weight
class_weight = compute_class_weight(class_weight='balanced',
                                    classes=np.unique(cls_train),
                                    y=cls_train)
print(class_weight)

[0.86347518 1.22979798 0.75154321 1.1763285  0.9898374  1.19362745]


In [0]:
#train new vgg model (transfer learning)
all_history =[]
for i in range(100):
    history = new_model.fit_generator(
            generator=generator_train,
            epochs=epochs,
            steps_per_epoch=steps_per_epoch,
            class_weight=class_weight,
            validation_data=generator_test)
    generator_test.reset()
    result = loaded_vgg_model.evaluate_generator(generator_test, steps=steps_test)
    print('result: ',result)
    if i%10==0:
        new_model.save('/content/drive/My Drive/Colab Notebooks/face recognition/model.h5')

Epoch 1/2
100/100 [==============================] - 41s 408ms/step - loss: 1.7766 - categorical_accuracy: 0.2416 - val_loss: 1.7369 - val_categorical_accuracy: 0.6122
Epoch 2/2
100/100 [==============================] - 39s 392ms/step - loss: 1.7452 - categorical_accuracy: 0.3957 - val_loss: 1.6892 - val_categorical_accuracy: 0.6122
result:  [1.1913758240589478, 0.8027210819477938]
Epoch 1/2
100/100 [==============================] - 41s 412ms/step - loss: 1.7133 - categorical_accuracy: 0.4281 - val_loss: 1.6382 - val_categorical_accuracy: 0.5986
Epoch 2/2
100/100 [==============================] - 39s 390ms/step - loss: 1.6795 - categorical_accuracy: 0.4496 - val_loss: 1.5855 - val_categorical_accuracy: 0.6190
result:  [1.1913758240589478, 0.8027210819477938]
Epoch 1/2
100/100 [==============================] - 41s 407ms/step - loss: 1.6457 - categorical_accuracy: 0.4643 - val_loss: 1.5339 - val_categorical_accuracy: 0.6190
Epoch 2/2
100/100 [==============================] - 39s 390

In [11]:
print('loading model')
loaded_vgg_model = load_model('/content/drive/My Drive/Colab Notebooks/face recognition/model.h5')
print('model loaded')

loading model



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


model loaded


In [12]:
loaded_vgg_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_2 (Model)              (None, 2622)              145002878 
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              2685952   
_________________________________________________________________
dropout_6 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 6150      
Total params: 147,694,980
Trainable params: 2,692,102
Non-trainable params: 145,002,878
_________________________________________________________________


In [0]:
print_layer_trainable(loaded_vgg_model)
for layer in loaded_vgg_model.layers:
    layer.trainable = False
for layer in loaded_vgg_model.layers[-17:]:
    layer.trainable = True
print('after unfreazing')
print_layer_trainable(loaded_vgg_model)

In [0]:
#traning new model (file tuning)
all_history =[]
count =0
for i in range(100):
    history=loaded_vgg_model.fit_generator(
            generator=generator_train,
            epochs=epochs,
            steps_per_epoch=steps_per_epoch,
            class_weight=class_weight,
            validation_data=generator_test)
    all_history.append(history)
    generator_test.reset()
    result = loaded_vgg_model.evaluate_generator(generator_test, steps=steps_test)
    print('result: ',result)
    if i%10==0:
        loaded_vgg_model.save('/content/drive/My Drive/Colab Notebooks/face recognition/tuned_model.h5')

In [4]:
with tf.device('/cpu:0'):#this is done because my gpu dont have enough memory for the model
    print('loading model')
    loaded_vgg_model = load_model('./model.h5')
    print('model loaded')
    loaded_vgg_model.summary()

loading model
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
model loaded
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_2 (Model)              (None, 2622)              145002878 
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              2685952   
_________________________________________________________________
dropout_6 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 6150      
Total params: 147,694,980
Trainable params: 2,692,102
Non-trainable params: 145,002,878
_________________________________________________________________


In [5]:
# def softmax(x):
#     """Compute softmax values for each sets of scores in x."""
#     return np.exp(x) / np.sum(np.exp(x), axis=0)
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

scores = [3.0, 1.0, 0.2]
print(softmax(scores))

[0.8360188  0.11314284 0.05083836]


In [10]:
#recognising faces
color = (255,255,0)
cap = cv2.VideoCapture(0) 
#try:
while(True):
    ret, img = cap.read()
    #img = cv2.imread(img)
    #print(img.shape())
    if ret:
        faces = detector.detect_faces(img)
        print('faces',faces)
        for face in faces:
            print('finding box')
            x,y,w,h = face['box']
            if w > 130: 
                detected_face = img[int(y):int(y+h), int(x):int(x+w)] #crop detected face
                detected_face = cv2.resize(detected_face, (224, 224)) #resize to 224x224

                img_pixels = image.img_to_array(detected_face)
                img_pixels = np.expand_dims(img_pixels, axis = 0)

                img_pixels /= 127.5
                img_pixels -= 1

                captured_representation = loaded_vgg_model.predict(img_pixels)

                score = softmax(captured_representation[0])
                confidence = max(score)
                y_classes = captured_representation.argmax(axis=-1)
                text = class_names[y_classes]
                if confidence>.3:
                    text = class_names[y_classes]
                else:
                    text = 'unknown'
                org = (int(x+w+15), int(y-12))
                check = cv2.putText(img=img, 
                                    text=text[0], 
                                    org= org, 
                                    fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                                    fontScale= 1, 
                                    color= color, 
                                    thickness= 1, 
                                    lineType= 2)
                cv2.line(img,(int((x+x+w)/2),y+15),(x+w,y-20),color,1)
                cv2.line(img,(x+w,y-20),(x+w+10,y-20),color,1)
        cv2.imshow('img',img)

    if cv2.waitKey(1) & 0xFF == ord('q'): #press q to quit
        break
        
cap.release()
cv2.destroyAllWindows()

faces [{'box': [275, 213, 118, 148], 'confidence': 0.9986976385116577, 'keypoints': {'left_eye': (313, 270), 'right_eye': (370, 272), 'nose': (342, 305), 'mouth_left': (316, 332), 'mouth_right': (364, 333)}}]
finding box
faces [{'box': [275, 215, 120, 150], 'confidence': 0.9984908103942871, 'keypoints': {'left_eye': (312, 273), 'right_eye': (371, 273), 'nose': (342, 307), 'mouth_left': (318, 335), 'mouth_right': (366, 335)}}]
finding box
faces [{'box': [275, 215, 118, 150], 'confidence': 0.9986555576324463, 'keypoints': {'left_eye': (312, 272), 'right_eye': (370, 273), 'nose': (341, 307), 'mouth_left': (317, 335), 'mouth_right': (365, 336)}}]
finding box
faces [{'box': [275, 213, 118, 146], 'confidence': 0.9982718229293823, 'keypoints': {'left_eye': (315, 267), 'right_eye': (370, 268), 'nose': (344, 302), 'mouth_left': (315, 326), 'mouth_right': (364, 328)}}]
finding box
faces [{'box': [276, 215, 117, 146], 'confidence': 0.9985895752906799, 'keypoints': {'left_eye': (314, 270), 'right_

faces [{'box': [269, 165, 132, 166], 'confidence': 0.9984027743339539, 'keypoints': {'left_eye': (312, 231), 'right_eye': (377, 229), 'nose': (349, 266), 'mouth_left': (319, 299), 'mouth_right': (373, 298)}}]
finding box
captured_representation  [[0.016983   0.00984592 0.00427092 0.01034084 0.00340404 0.95515525]]
softmax [0.13305913 0.13211285 0.13137837 0.13217825 0.13126454 0.34000686]
confidence 0.34000686
y class [5]
class name ['kaushal']
<class 'numpy.ndarray'>
['kaushal']
<class 'tuple'>
<class 'int'>
<class 'tuple'>
<class 'int'>
<class 'numpy.ndarray'>
faces [{'box': [272, 155, 132, 167], 'confidence': 0.995501697063446, 'keypoints': {'left_eye': (321, 219), 'right_eye': (383, 219), 'nose': (356, 256), 'mouth_left': (322, 286), 'mouth_right': (375, 287)}}]
finding box
captured_representation  [[0.0205081  0.01516714 0.00737953 0.01182901 0.00544917 0.93966705]]
softmax [0.13395445 0.13324091 0.13220732 0.13279688 0.13195236 0.33584803]
confidence 0.33584803
y class [5]
class 

faces [{'box': [258, 163, 136, 168], 'confidence': 0.9965993762016296, 'keypoints': {'left_eye': (306, 230), 'right_eye': (370, 225), 'nose': (344, 266), 'mouth_left': (313, 298), 'mouth_right': (368, 294)}}]
finding box
captured_representation  [[0.02333441 0.01820924 0.00676242 0.01706952 0.0077562  0.92686826]]
softmax [0.13467905 0.13399056 0.13246554 0.13383794 0.13259725 0.3324297 ]
confidence 0.3324297
y class [5]
class name ['kaushal']
<class 'numpy.ndarray'>
['kaushal']
<class 'tuple'>
<class 'int'>
<class 'tuple'>
<class 'int'>
<class 'numpy.ndarray'>
faces [{'box': [259, 165, 135, 167], 'confidence': 0.99483722448349, 'keypoints': {'left_eye': (306, 230), 'right_eye': (370, 225), 'nose': (344, 265), 'mouth_left': (313, 297), 'mouth_right': (368, 294)}}]
finding box
captured_representation  [[0.02202969 0.02476999 0.00695183 0.01594389 0.00944307 0.9208615 ]]
softmax [0.13466316 0.1350327  0.13264796 0.13384613 0.13297883 0.33083126]
confidence 0.33083126
y class [5]
class na

captured_representation  [[0.00850575 0.00675002 0.00352818 0.00533729 0.00179437 0.9740844 ]]
softmax [0.13141061 0.1311801  0.13075814 0.1309949  0.13053162 0.3451246 ]
confidence 0.3451246
y class [5]
class name ['kaushal']
<class 'numpy.ndarray'>
['kaushal']
<class 'tuple'>
<class 'int'>
<class 'tuple'>
<class 'int'>
<class 'numpy.ndarray'>
faces []
faces []
faces [{'box': [277, 181, 136, 167], 'confidence': 0.9994146823883057, 'keypoints': {'left_eye': (321, 248), 'right_eye': (386, 248), 'nose': (356, 287), 'mouth_left': (326, 317), 'mouth_right': (381, 317)}}]
finding box
captured_representation  [[0.00811363 0.00582493 0.00188953 0.00472533 0.00159729 0.97784925]]
softmax [0.13125291 0.13095286 0.13043852 0.13080893 0.1304004  0.34614637]
confidence 0.34614637
y class [5]
class name ['kaushal']
<class 'numpy.ndarray'>
['kaushal']
<class 'tuple'>
<class 'int'>
<class 'tuple'>
<class 'int'>
<class 'numpy.ndarray'>
faces [{'box': [274, 182, 133, 164], 'confidence': 0.9983754158020